# Read and dump raw ACTL header and event

__calin__ provides some level of access to the raw ACTL header and event protobuf structures. In Python it is possible to retrieve the header and any desired event from ZFits but __calin__ does not provide simple access to the fields in the protobuf in the usual way. However the messages can be serialized to JSON and then accessed through the Python JSON packages if desired. This is only really intended for debugging.

In [ ]:
%pylab inline
import calin.iact_data.raw_actl_event_data_source
import json
import struct
import base64

## 1 - Create ACTL source

In [ ]:
actl_src = calin.iact_data.raw_actl_event_data_source.ZFITSACTLDataSource(
    '/CTA/cta.cppm.in2p3.fr/NectarCAM/20161207/Run0268.1.fits.fz')

## 2 - Read header and print JSON string

In [ ]:
header = actl_src.get_run_header()
print(header.SerializeAsJSON())

## 3 - Read first event and print JSON string

In [ ]:
event = actl_src.simple_get_next()
str_event = event.SerializeAsJSON();
print(str_event)

## 4 - Use Python JSON decoder to conver string to Python structure

Print keys of the JSON dictionary

In [ ]:
json_event = json.loads(str_event)
json_event.keys()

## 5 - Convert samples data to integer array

The bytes array from the ACTL Protobuf structure is encoded as a base64 string in the JSON; for details see the mapping guide below:

https://developers.google.com/protocol-buffers/docs/proto3#json

In [ ]:
nsamples = json_event['hiGain']['waveforms']['numSamples']
samples_data = base64.b64decode(json_event['hiGain']['waveforms']['samples']['data'])
samples = frombuffer(samples_data,dtype='int16').reshape([-1,nsamples])
plot(samples.transpose())
xlabel('Sample number')
ylabel('Sample amplitude [DC]')